In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, Flatten
import pickle

In [5]:
IoT = pd.read_csv('../../data/IoT_Weather.csv')
IoT = IoT.iloc[0:500]

IoT = IoT.dropna()

encoder=LabelEncoder()
IoT['type']=encoder.fit_transform(IoT['type'])
# IoT['state']=encoder.fit_transform(IoT['state'])
# IoT['sphone_signal']=encoder.fit_transform(IoT['sphone_signal'])


IoT.head()

,date,time,temperature,pressure,humidity,label,type
0,31-Mar-19,12:36:52,31.788508,1.035,32.036579,0,0
1,31-Mar-19,12:36:53,41.630997,1.035,30.886165,0,0
2,31-Mar-19,12:36:54,42.256959,1.035,19.755908,0,0
3,31-Mar-19,12:36:55,49.116581,1.035,78.949621,0,0
4,31-Mar-19,12:36:56,24.017085,1.035,40.001059,0,0


In [6]:
x = IoT.drop(['label', 'date', 'time','type'], axis=1)
y = IoT['label']

features=len(x.columns)

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

In [7]:
model2 = Sequential()
    
model2.add(GRU(units = 128, activation='tanh', return_sequences=True, input_shape = (features,1)))
model2.add(Dropout(0.2))
model2.add(GRU(units = 64, activation='tanh', return_sequences=True))
model2.add(Dropout(0.2))
model2.add(Dense(1, activation='sigmoid')) 
model2.add(Flatten())
    
model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
model2.save('../h5s/GRU.h5') 
# filename = '../h5s/GRU.h5'
# pickle.dump(model2, open(filename, 'wb'))

In [9]:
model2.fit(x_train, np.asarray(y_train).astype('float32').reshape((-1,1)), epochs = 5, batch_size=32, validation_data=(x_test, np.asarray(y_test).astype('float32').reshape((-1,1))))

Epoch 1/5
13/13 [==============================] - 6s 81ms/step - loss: 0.1093 - accuracy: 0.9425 - val_loss: 0.0171 - val_accuracy: 1.0000
Epoch 2/5
13/13 [==============================] - 0s 9ms/step - loss: 0.0129 - accuracy: 0.9975 - val_loss: 0.0054 - val_accuracy: 1.0000
Epoch 3/5
13/13 [==============================] - 0s 9ms/step - loss: 0.0047 - accuracy: 0.9975 - val_loss: 0.0022 - val_accuracy: 1.0000
Epoch 4/5
13/13 [==============================] - 0s 9ms/step - loss: 0.0021 - accuracy: 0.9950 - val_loss: 9.9214e-04 - val_accuracy: 1.0000
Epoch 5/5
13/13 [==============================] - 0s 9ms/step - loss: 9.7208e-04 - accuracy: 0.9950 - val_loss: 4.8134e-04 - val_accuracy: 1.0000


In [10]:
predictions = model2.predict(x_test)
y_pred = [round(x[0]) for x in predictions]
y_pred

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [11]:
for i in range(3):
	print(x_test.values[i].tolist(), '=>', y_pred[i], '( expected ', y_train.values[i], ')')

[47.24837461, 1.035, 44.26911976] => 0 ( expected  0 )
[40.44845884, 1.142511755, 8.070560667] => 0 ( expected  0 )
[26.16359086, 0.694716943, 58.86706826] => 0 ( expected  0 )


In [12]:
score, acc = model2.evaluate(x_test, y_test, batch_size=1)

print('Test score:', score)
print('Test accuracy:', acc)

100/100 [==============================] - 1s 2ms/step - loss: 4.8133e-04 - accuracy: 1.0000
Test score: 0.0004813330015167594
Test accuracy: 1.0
